In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile, os

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

zip_id = '1upOcakmV4Ym2sv-5K3MY88rA_QNzTXJV'
print ("Downloading zip file")
myzip = drive.CreateFile({'id': zip_id})
myzip.GetContentFile('model.zip')
print ("Uncompressing zip file")
zip_ref = zipfile.ZipFile('model.zip', 'r')
zip_ref.extractall("breast_density_dataset")
zip_ref.close()

In [0]:
import pandas as pd
valid = pd.read_csv("breast_density_dataset/train.csv",nrows=2000)
train = pd.read_csv("breast_density_dataset/train.csv", skiprows=2000)
train.columns = valid.columns

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in valid.columns[2:-1]:
  valid[col]=le.fit_transform(valid[col])
  train[col]=le.fit_transform(train[col])

train =train.sample(frac=1).reset_index(drop=True)
valid =valid.sample(frac=1).reset_index(drop=True)
train.to_csv("train.csv",index=False)
valid.to_csv("valid.csv",index=False)

In [0]:
import pandas as pd
import numpy as np
import os
import h5py
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

def read_image_and_breast_mask(h5_filename):
    # 'r' mode is very important ! 
    with h5py.File(h5_filename, "r") as h5_file:
        image = h5_file["image"][:]
        breast_mask = h5_file["mask"][:]

    return image, breast_mask

In [0]:
def batch_generator(Train_df,batch_size,steps):
 idx=1
 while True: 
  yield load_data(Train_df,idx-1,batch_size)## Yields data
  if idx<steps:
    idx+=1
  else:
    idx=1

In [0]:
TRAIN_DIR = "breast_density_dataset/train/"
TEST_DIR = "breast_density_dataset/test/"

def load_data(Train_df,idx,batch_size):
 df = pd.read_csv(Train_df, skiprows=idx*batch_size,nrows=batch_size)
 x = []
 z = []
 y = [] 
 for  row  in df.values:
  image_uid=row[0]
  image_file = os.path.join(TRAIN_DIR, image_uid + ".h5")
  X, breast_mask = read_image_and_breast_mask(image_file)

  image = (X - X.min()) / (X.max() - X.min())
  #image =(image*255.).astype('uint8')
  
 # tmp=np.zeros(2)
 # tmp[row[-2]-1]=1
  #latent=np.array(functors([image.reshape(1,288, 208, 1), 1.])).reshape(36, 26, 128)
  #z.append(row[2:-1])
  x.append(image) 
  y.append(row[2:] )

 return (np.array(x).reshape(-1,288,208,1), np.array(y).reshape(-1,5))

In [0]:
import numpy as np
batch_size=32
steps_per_epoch=np.ceil(7960/batch_size)
validation_steps=np.ceil(2000/batch_size)

my_training_batch_generator = batch_generator('train.csv', 32,steps_per_epoch)
my_validation_batch_generator = batch_generator('valid.csv', 32,validation_steps)

In [75]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D,Flatten,Reshape,Convolution2D,Cropping2D
from keras.models import Model
from skimage.transform import resize
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau



input_img = Input(shape=(288,208,1))  # adapt this if using `channels_first` image data format

x = Conv2D(32, (5, 5), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(64, (5, 5), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(128, (5, 5), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)


x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(64, (5, 5), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(128, (5, 5), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)



x=Flatten()(x)
x = Dense(1024)(x)

x = Dense(512)(x)

x = Dense(256)(x)

output = Dense(5,activation='relu')(x)


 



model = Model(input_img, [output])
model.compile(optimizer='adam', loss=['mse'])
model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 288, 208, 1)       0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 288, 208, 32)      832       
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 144, 104, 32)      0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 144, 104, 64)      51264     
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 72, 52, 64)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 72, 52, 128)       204928    
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 36, 26, 128)       0  

In [0]:
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-s.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [77]:
results = model.fit_generator(my_training_batch_generator,epochs=30,steps_per_epoch=steps_per_epoch,verbose=1, validation_data=my_validation_batch_generator,validation_steps=validation_steps, callbacks=callbacks)

Epoch 1/30
249/249 [==============================] - 32s 130ms/step - loss: 0.9357 - val_loss: 0.6937

Epoch 00001: val_loss improved from inf to 0.69367, saving model to model-s.h5
Epoch 2/30
249/249 [==============================] - 28s 114ms/step - loss: 0.6732 - val_loss: 0.5437

Epoch 00002: val_loss improved from 0.69367 to 0.54365, saving model to model-s.h5
Epoch 3/30
249/249 [==============================] - 29s 116ms/step - loss: 0.5498 - val_loss: 0.5263

Epoch 00003: val_loss improved from 0.54365 to 0.52626, saving model to model-s.h5
Epoch 4/30
249/249 [==============================] - 29s 115ms/step - loss: 0.5382 - val_loss: 0.5194

Epoch 00004: val_loss improved from 0.52626 to 0.51940, saving model to model-s.h5
Epoch 5/30
249/249 [==============================] - 29s 116ms/step - loss: 0.5297 - val_loss: 0.5113

Epoch 00005: val_loss improved from 0.51940 to 0.51130, saving model to model-s.h5
Epoch 6/30
249/249 [==============================] - 29s 118ms/step 

In [78]:
df = pd.read_csv("train.csv")
s=0
for  row  in df.values:
  image_uid=row[0]
  image_file = os.path.join(TRAIN_DIR, image_uid + ".h5")
  X, breast_mask = read_image_and_breast_mask(image_file)
  image = (X - X.min()) / (X.max() - X.min())
  tmp=model.predict(image.reshape(1,288, 208, 1))
  p=np.abs(np.round(tmp.reshape(-1)))
  if p[-1]==row[-1]:
    s+=1
print("precision :"+str(s/len(df)))

precision :0.821608040201005


In [0]:
test = pd.read_csv("breast_density_dataset/train.csv")
y_pred=[]
for  row  in test.values:
  image_uid=row[0]
  image_file = os.path.join(TRAIN_DIR, image_uid + ".h5")
  X, breast_mask = read_image_and_breast_mask(image_file)
  image = (X - X.min()) / (X.max() - X.min())
  tmp=model.predict(image.reshape(1,288, 208, 1))
  p=np.abs(np.round(tmp.reshape(-1)))
  y_pred.append(int(p[-1]))

test["breast_density_score"]=y_pred

In [80]:
test.head()

,image_uid,study_uid,source,manufacturer,view,laterality,breast_density_score
0,pJKpQYcIUh,zHcZzUxPOaEdmbhUSHId,BANI,A,CC,L,2
1,iemeEWCujY,zHcZzUxPOaEdmbhUSHId,BANI,A,MLO,L,3
2,YmTbYwFFVD,zHcZzUxPOaEdmbhUSHId,BANI,A,CC,R,2
3,WwZyDUkXle,zHcZzUxPOaEdmbhUSHId,BANI,A,MLO,R,2
4,mOFgddQiUt,ljhFclzlCrLgXrDLkIeS,BANI,C,CC,L,1


In [0]:
test.to_csv("submissions.csv",index=False)